In [40]:
import datetime
import sys
from yahoo_finance_api2 import share
from yahoo_finance_api2.exceptions import YahooFinanceError

import matplotlib.pyplot as plt

import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
import math

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [41]:
stocks = ['F', 
          'GE', 
          'AAL', 
          'DIS', 
          'DAL', 
          'AAPL', 
          'MSFT', 
          'CCL', 
          'GPRO', 
          'ACB', 
          'PLUG', 
          'NCLH',
          'SNAP',
          'BAC',
          'BA',
          'FIT',
          'UAL',
          'MRNA',
          'NIO',
          'UBER',
          'BABA',
          'AMD',
          'FB',
          'CGC',
          'TWTR',
          'RCL',
          'NFLX',
          'CRON',
          'ZNGA',
          'T',
          'KO',
          'INO',
          'SAVE',
          'APHA',
          'SBUX',
          'LUV',
          'PFE',
          'JBLU',
          'NKLA',
          'MRO',
          'MGM',
          'GNUS',
          'XOM',
          'MFA',
          'GUSH',
          'SPCE',
          'USO',
          'NVDA',
          'AMC']


In [13]:
my_share = share.Share('NCLH')
symbol_data = None


symbol_data = pd.DataFrame.from_dict(my_share.get_historical(share.PERIOD_TYPE_DAY,
                                          1,
                                          share.FREQUENCY_TYPE_MINUTE,
                                          1)).dropna()

In [14]:
symbol_data.shape

(621, 6)

In [36]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 1]
        dataX.append(a)
        dataY.append(1 if dataset[i + look_back, 1] > dataset[i + look_back-1, 1] else 0)
    return np.array(dataX), np.array(dataY)

In [56]:
look_back = 10
dataX, dataY = create_dataset(symbol_data[['timestamp', 'close']].values, look_back)
X_train, X_test, y_train, y_test = train_test_split(
     dataX, dataY, test_size=0.33, random_state=42)
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

In [62]:
model = Sequential()
model.add(LSTM(4, input_shape=(1, look_back)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=1, verbose=2)

Train on 408 samples, validate on 202 samples
Epoch 1/20
 - 2s - loss: 0.2638 - val_loss: 0.2445
Epoch 2/20
 - 1s - loss: 0.2478 - val_loss: 0.2439
Epoch 3/20
 - 1s - loss: 0.2474 - val_loss: 0.2448
Epoch 4/20
 - 1s - loss: 0.2464 - val_loss: 0.2438
Epoch 5/20
 - 1s - loss: 0.2461 - val_loss: 0.2438
Epoch 6/20
 - 1s - loss: 0.2450 - val_loss: 0.2437
Epoch 7/20
 - 1s - loss: 0.2451 - val_loss: 0.2438
Epoch 8/20
 - 1s - loss: 0.2449 - val_loss: 0.2439
Epoch 9/20
 - 1s - loss: 0.2449 - val_loss: 0.2438
Epoch 10/20
 - 1s - loss: 0.2452 - val_loss: 0.2438
Epoch 11/20
 - 1s - loss: 0.2447 - val_loss: 0.2443
Epoch 12/20
 - 1s - loss: 0.2451 - val_loss: 0.2442
Epoch 13/20
 - 1s - loss: 0.2447 - val_loss: 0.2438
Epoch 14/20
 - 1s - loss: 0.2450 - val_loss: 0.2438
Epoch 15/20
 - 1s - loss: 0.2446 - val_loss: 0.2439
Epoch 16/20
 - 1s - loss: 0.2446 - val_loss: 0.2437
Epoch 17/20
 - 1s - loss: 0.2448 - val_loss: 0.2438
Epoch 18/20
 - 1s - loss: 0.2445 - val_loss: 0.2437
Epoch 19/20
 - 1s - loss: 0

In [65]:
scores = model.evaluate(X_test, y_test, verbose=0)
print(scores)

0.24384675418386365


In [60]:
X_train.shape

(408, 1, 10)